In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/sentiment-analysis/Sentiment_Analysis.csv


In [5]:
df=pd.read_csv("/kaggle/input/sentiment-analysis/Sentiment_Analysis.csv")

In [6]:
df

,sentiment,text
0,1,"And here is the rap song ""African Warrior Quee..."
1,1,"We asked chatGPT ,\n\nHow to become a successf..."
2,0,When I finally manage to convince myself to pe...
3,0,Although a healthier alternative to your typic...
4,1,"In my latest test, I asked #ChatGPT to write m..."
...,...,...
79995,1,This movie just might make you cooooo. The fil...
79996,0,Stack Overflow temporarily bans answers from O...
79997,0,"<span class=""tiny""> Length:: 0:49 Mins<br /><b..."
79998,1,If it is true that sadomasochism is a two-side...


In [3]:
!pip install spacy
!python -m spacy download en_core_web_md

INFO: pip is looking at multiple versions of mkl-fft to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of mkl-random to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of mkl-umath to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.6/16.6 MB 70.9 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
mkl-umath 0.1.1 requires numpy<1.27.0,>=1.26.4, but

In [4]:
import spacy
nlp = spacy.load("en_core_web_md")


In [7]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test= train_test_split(df['text'],df['sentiment'], test_size=0.3)

In [9]:
import numpy as np

In [11]:
from tqdm import tqdm

X_train_vec = np.array([
    nlp(text).vector
    for text in tqdm(X_train, desc="Vectorizing train data")
])

X_test_vec = np.array([
    nlp(text).vector
    for text in tqdm(X_test, desc="Vectorizing test data")
])


Vectorizing test data: 100%|██████████| 24000/24000 [10:57<00:00, 36.51it/s]


In [12]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(max_iter=1000)
clf.fit(X_train_vec, y_train)

y_pred = clf.predict(X_test_vec)


In [13]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       0.69      0.70      0.69     11997
           1       0.69      0.68      0.69     12003

    accuracy                           0.69     24000
   macro avg       0.69      0.69      0.69     24000
weighted avg       0.69      0.69      0.69     24000



In [14]:
!pip install xgboost


In [15]:
from xgboost import XGBClassifier

xgb = XGBClassifier(
    n_estimators=500,
    learning_rate=0.05,
    max_depth=6,
    subsample=0.8,
    colsample_bytree=0.8,
    objective="binary:logistic",
    eval_metric="logloss",
    random_state=42
)

xgb.fit(X_train_vec, y_train)

y_pred_xgb = xgb.predict(X_test_vec)

In [16]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred_xgb))

              precision    recall  f1-score   support

           0       0.71      0.72      0.71     11997
           1       0.72      0.71      0.71     12003

    accuracy                           0.71     24000
   macro avg       0.71      0.71      0.71     24000
weighted avg       0.71      0.71      0.71     24000



In [17]:
from sklearn.svm import LinearSVC

svm = LinearSVC()
svm.fit(X_train_vec, y_train)

y_pred = svm.predict(X_test_vec)


/usr/local/lib/python3.11/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [18]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.69      0.70      0.69     11997
           1       0.70      0.69      0.69     12003

    accuracy                           0.69     24000
   macro avg       0.69      0.69      0.69     24000
weighted avg       0.69      0.69      0.69     24000

